#### Compute adjacency complex to track persistence

In [17]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import pickle as pickle
from pylab import *
import os    
import numpy as np

import warnings

import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import itertools
from itertools import combinations
from scipy import spatial
import pickle as pickle
import gudhi
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
import io
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageChops, ImageFont
import shapely.geometry as geom
from shapely.ops import unary_union

import invr

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [18]:
# get the list of folders in a location
def get_folders(location):
    return [name for name in os.listdir(location) if os.path.isdir(os.path.join(location, name))]

In [19]:
states = get_folders('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes')

In [20]:
print('Number of states:', len(states))
print(states)

Number of states: 50
['VT', 'VA', 'SD', 'SC', 'UT', 'GA', 'MS', 'MT', 'MO', 'MA', 'AK', 'KY', 'AL', 'NH', 'MN', 'MI', 'OK', 'IN', 'CO', 'CA', 'IA', 'CT', 'FL', 'WV', 'RI', 'WY', 'TX', 'PA', 'NC', 'ND', 'NM', 'NJ', 'ME', 'AR', 'NV', 'MD', 'KS', 'NE', 'HI', 'DE', 'AZ', 'NY', 'ID', 'OH', 'OR', 'IL', 'LA', 'WI', 'WA', 'TN']


In [21]:
for state in states:
    # create a folder for each state if it does not exist
    os.makedirs(f"/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/selected coordinates for each state - percentiles(below 90th)- all variables/{state}", exist_ok=True)
print('Done creating folders for each state')

Done creating folders for each state


In [30]:
selected_variables = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ','NOD_Rate']
selected_variables_without_y = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ']

# selected_variables_for_state_with_geo = ['FIPS','EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV','NOD_Rate','geometry']
# selected_variables_for_state = ['EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV']
# selected_variables_tn_with_od = ['EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV','NOD_Rate']

selected_variables_for_state_with_geo = ['FIPS','EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ','NOD_Rate','geometry']
selected_variables_for_state = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ']
selected_variables_tn_with_od = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ','NOD_Rate']

selected_variables_with_censusinfo = ['FIPS','STCNTY','EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ','NOD_Rate','geometry']


In [37]:
# def generate_adjacent_counties(dataframe,filtration_threshold,variable_name):
def generate_adjacent_counties(dataframe,variable_name):


    filtered_df = dataframe
    # filtered_df = dataframe[dataframe[variable_name] < filtration_threshold]

    # Perform a spatial join to find adjacent precincts
    adjacent_counties = gpd.sjoin(filtered_df, filtered_df, predicate='intersects', how='left')

    # Filter the results to include only the adjacent states
    adjacent_counties = adjacent_counties.query('sortedID_left != sortedID_right')

    # Group the resulting dataframe by the original precinct Name and create a list of adjacent precinct Name
    adjacent_counties = adjacent_counties.groupby('sortedID_left')['sortedID_right'].apply(list).reset_index()

    adjacent_counties.rename(columns={'sortedID_left': 'county', 'sortedID_right': 'adjacent'}, inplace=True)

    adjacencies_list = adjacent_counties['adjacent'].tolist()
    county_list = adjacent_counties['county'].tolist()

    merged_df = pd.merge(adjacent_counties, dataframe, left_on='county',right_on='sortedID', how='left')
    merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

    return adjacencies_list,merged_df,county_list

In [24]:
def form_simplicial_complex(adjacent_county_list,county_list):
    max_dimension = 3

    V = []
    V = invr.incremental_vr(V, adjacent_county_list, max_dimension,county_list)

    return V

In [25]:
def plot_simplicial_complex(dataframe,V):

    #city centroids
    city_coordinates = {city.sortedID: np.array((city.geometry.centroid.x, city.geometry.centroid.y)) for _, city in dataframe.iterrows()}

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_axis_off() 

    # Plot the "wyoming_svi" DataFrame
    dataframe.plot(ax=ax, edgecolor='black', linewidth=0.3, color="white")

    # Plot the centroid of the large square with values
    # for i, row in dataframe.iterrows():
    #     centroid = row['geometry'].centroid
    #     # text_to_display = f"FIPS: {row['FIPS']}\nFilteration: {row['EP_SNGPNT']}"
    #     plt.text(centroid.x, centroid.y, str(row['FIPS']), fontsize=8, ha='center', color="black")
    #     # plt.text(centroid.x, centroid.y, text_to_display, fontsize=10, ha='center', color="black")

    for edge_or_traingle in V:

        
        if len(edge_or_traingle) == 2:
            # Plot an edge
            ax.plot(*zip(*[city_coordinates[vertex] for vertex in edge_or_traingle]), color='red', linewidth=1)
            # img = fig2img(fig)
            # list_gif.append(img)
        elif len(edge_or_traingle) == 3:
            # Plot a triangle
            ax.add_patch(plt.Polygon([city_coordinates[vertex] for vertex in edge_or_traingle], color='green', alpha=0.2))
            # img = fig2img(fig)
            # list_gif.append(img)
    # plt.show()
    plt.close()

In [39]:
for state in states:

    print('Processing:', state)

    # Initialize dictionaries to store the percentiles
    


    svi_od = gpd.read_file(f'/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes/{state}/{state}.shp')
    
    # Drop -999 in selected_variables_for_state
    for variable in selected_variables_for_state:
        svi_od = svi_od[svi_od[variable] != -999]

    

    svi_od_filtered_state = svi_od[selected_variables_with_censusinfo]

    #reset index
    svi_od_filtered_state = svi_od_filtered_state.reset_index(drop=True)

    # get the uniques counties
    unique_county_stcnty = svi_od_filtered_state['STCNTY'].unique()

    print('Number of counties:', len(unique_county_stcnty))


    # percentiles_50 = {}
    # percentiles_75 = {}
    # percentiles_90 = {}

    for county_stcnty in unique_county_stcnty:
        print('Processing:', county_stcnty)

        # Filter the dataframe to include only the current county
        county_svi_df = svi_od_filtered_state[svi_od_filtered_state['STCNTY'] == county_stcnty]

        print('Number of rows:', len(county_svi_df))

        # # Calculate the percentiles for each variable
            # for variable in selected_variables_for_state:
            #     percentiles_50[variable] = svi_od[variable].quantile(0.5)
            #     percentiles_75[variable] = svi_od[variable].quantile(0.75)
            #     percentiles_90[variable] = svi_od[variable].quantile(0.9)

        # need to change here later

        for variable_name in selected_variables_without_y:
            
            print('Processing:', variable)

            # Sorting based on the variable and selecting only the FIPS and the variable columns is important
            # Also we need to keep  the dataframe sorted based on the variable


            # Cahanged FIPS into STCNTY
            df_one_variable = county_svi_df[['STCNTY',variable_name, 'geometry']]

            # # Sorting the DataFrame based on the 'rate' column
            df_one_variable = df_one_variable.sort_values(by=variable_name)
            df_one_variable['sortedID'] = range(len(df_one_variable))

            # Convert the DataFrame to a GeoDataFrame
            df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
            df_one_variable.crs = "EPSG:3395"  # This is a commonly used projected CRS

            # Heter thresho;d removed from original adj method - change it later
            adjacencies_list,adjacent_counties_df,county_list = generate_adjacent_counties(df_one_variable,variable_name)






        break
    break


print('All states processed.')

Processing: VT
Number of counties: 14
Processing: 50001
Number of rows: 10
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
Processing: EP_GROUPQ
All states processed.
